#### SMOLVLA MODEL INFERENCE

#### 1.Library

In [1]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
import numpy as np
from lerobot.common.datasets.utils import write_json, serialize_dict
from lerobot.common.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.common.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.configs.types import FeatureType
from lerobot.common.datasets.factory import resolve_delta_timestamps
from lerobot.common.datasets.utils import dataset_to_policy_features
import torch
from PIL import Image
import torchvision

/home/oguz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Loading Configs & Policy

In [2]:
# # Install Dataset if doesn't exist
# !git clone https://huggingface.co/datasets/Jeongeun/omy_pnp_language

In [3]:
from torchvision import transforms
def get_default_transform(image_size: int = 224):
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(), 
    ])

In [4]:
dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./omy_pnp_language')
features = dataset_to_policy_features(dataset_metadata.features)

output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}
input_features = {key: ft for key, ft in features.items() if key not in output_features}

cfg = SmolVLAConfig(input_features=input_features, output_features=output_features, chunk_size= 5, n_action_steps=5)
delta_timestamps = resolve_delta_timestamps(cfg, dataset_metadata)

In [5]:
policy = SmolVLAPolicy.from_pretrained(
    "Jeongeun/omy_pnp_smolvla", 
    config=cfg, 
    dataset_stats=dataset_metadata.stats
)

Reducing the number of VLM layers to 16 ...


#### 3. Setup Sim Environment

In [7]:
%pwd

'/home/oguz/Desktop/smolvla/embodied_ai/smolvla_project'

In [8]:
from mujoco_env.y_env2 import SimpleEnv2
xml_path = './asset/example_scene_y2.xml'
PnPEnv = SimpleEnv2(xml_path, action_type='joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [front_object_table] mass:[1.00]kg
 [2/23] [camera] mass:[0.00]kg
 [3/23] [camera2] mass:[0.00]kg
 [4/23] [camera3] mass:[0.00]kg
 [5/23] [link1] mass:[2.06]kg
 [6/23] [link2] mass:[3.68]kg
 [7/23] [link3] mass:[2.39]kg
 [8/23] [link4] mass:[1.40]kg
 [9/23] [link5] mass:[1.40]kg
 [10/23] [link6] mass:[0.65]kg
 [11/23] [camera_center] mass:[0.00]kg
 [12/23] [tcp_link] mass:[0.32]kg
 [13/23] [rh_p12_rn_r1] mass:[0.07]kg
 [14/23] [rh_p12_rn_r2] mass:[0.02]kg
 [15/23] [rh_p12_rn_l1] mass:[0.07]kg
 [16/23] [rh_p12_rn_l2] mass:[0.02]kg
 [17/23] [body_obj_mug_5] mass:[0.00]kg
 [18/23] [object_mug_5] mass:[0.08]kg
 [19/23] [body_obj_plate_11] mass:[0.00]kg
 [20/23] [object_plate_11] mass:[0.10]kg
 [21/23] [body_obj_mug_6] mass:[0.00]kg
 [22/23] [object_mug

#### 4. Run Evaluation

In [7]:
step = 0
device = "cuda"
save_image = True
IMG_TRANSFORM = get_default_transform()

PnPEnv.reset(seed=0)
policy.reset()

policy.eval()

while PnPEnv.env.is_viewer_alive():
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # Check if the task is completed
        success = PnPEnv.check_success()
        if success:
            print('Success')
            # Reset the environment and action queue
            policy.reset()
            PnPEnv.reset()
            step = 0
            save_image = False
        # Get the current state of the environment
        state = PnPEnv.get_joint_state()[:6]
        # Get the current image from the environment
        image, wirst_image = PnPEnv.grab_image()
        image = Image.fromarray(image)
        image = image.resize((256, 256))
        image = IMG_TRANSFORM(image)
        wrist_image = Image.fromarray(wirst_image)
        wrist_image = wrist_image.resize((256, 256))
        wrist_image = IMG_TRANSFORM(wrist_image)
        data = {
            'observation.state': torch.tensor([state]).to(device),
            'observation.image': image.unsqueeze(0).to(device),
            'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
            'task': [PnPEnv.instruction],
        }
        # Select an action
        action = policy.select_action(data)
        action = action[0,:7].cpu().detach().numpy()
        # Take a step in the environment
        _ = PnPEnv.step(action)
        PnPEnv.render()
        step += 1
        success = PnPEnv.check_success()
        if success:
            print('Success')
            break

DONE INITIALIZATION


/tmp/ipykernel_522459/3771870778.py:34: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  'observation.state': torch.tensor([state]).to(device),
